In [80]:
import os
import numpy as np
import pandas as pd

In [9]:
file_path = 'open_data'

In [54]:
# load data to dataframe
df_b, df_e, df_f, df_h = tuple([
    pd.read_csv(f"{file_path}/{file_name}", skiprows=[1]) for file_name in os.listdir(file_path)
])

In [55]:
print(f"{df_b.shape}, {df_e.shape}, {df_f.shape}, {df_h.shape}")

(1320, 28), (1129, 28), (1311, 28), (876, 28)


In [69]:
# merge df
df_all = pd.concat([df_b, df_e, df_f, df_h], ignore_index=True, axis=0)
print(df_all.shape)

(4636, 28)


In [74]:
df_all.columns

Index(['鄉鎮市區', '交易標的', '土地區段位置建物區段門牌', '土地移轉總面積平方公尺', '都市土地使用分區', '非都市土地使用分區',
       '非都市土地使用編定', '交易年月日', '交易筆棟數', '移轉層次', '總樓層數', '建物型態', '主要用途', '主要建材',
       '建築完成年月', '建物移轉總面積平方公尺', '建物現況格局-房', '建物現況格局-廳', '建物現況格局-衛',
       '建物現況格局-隔間', '有無管理組織', '總價元', '單價元平方公尺', '車位類別', '車位移轉總面積平方公尺', '車位總價元',
       '備註', '編號'],
      dtype='object')

In [226]:
# check unique field value
df_all.loc[:,'總樓層數'].unique()

array(['四層', '十九層', '七層', '十三層', '十二層', nan, '二層', '十五層', '十一層', '一層',
       '二十三層', '三層', '五層', '十四層', '六層', '二十五層', '八層', '十六層', '十層', '二十四層',
       '二十二層', '二十一層', '九層', '二十層', '二十八層', '三十二層', '十七層', '二十六層', '二十九層',
       '二十七層', '三十五層', '十八層', '三十九層', '八十五層', '三十六層', '四十三層', '三十一層',
       '四十一層', '三十七層', '三十四層', '四十層'], dtype=object)

In [222]:
def convert_building_num(building_count: str):
    """Convert chinese building floor to number."""

    ch_num_mapping = {
        "": 0, "一": 1, "二": 2, "三": 3,
        "四": 4, "五": 5, "六": 6, "七": 7,
        "八": 8, "九": 9, "十": 10,
    }
    
    if building_count is not np.nan:
        building_count = list(building_count.replace('層', ''))
        if len(building_count) == 1:
            building_count.insert(0, "")
            building_count.insert(0, "")
        elif (len(building_count) == 2) and (building_count[0] == '十'):
            building_count.insert(0, "一")    
        elif (len(building_count) == 2) and (building_count[0] != '十'):
            building_count.insert(2, "")
        building_count = [ch_num_mapping[count] for count in building_count]
        building_count = building_count[0] * building_count[1] + building_count[2]
    return building_count

In [224]:
df_all['building_floor_num'] = df_all['總樓層數'].map(convert_building_num)

In [230]:
df_all['building_floor_num'].unique()

array([ 4., 19.,  7., 13., 12., nan,  2., 15., 11.,  1., 23.,  3.,  5.,
       14.,  6., 25.,  8., 16., 10., 24., 22., 21.,  9., 20., 28., 32.,
       17., 26., 29., 27., 35., 18., 39., 85., 36., 43., 31., 41., 37.,
       34., 40.])

In [257]:
# filter_a.csv
filter_df_a = df_all.loc[df_all['building_floor_num']>=13,:]
filter_df_a = filter_df_a.loc[filter_df_a['主要用途']=='住家用',:]
filter_df_a = filter_df_a[filter_df_a['建物型態'].str.contains("住宅大樓")]
del filter_df_a['building_floor_num']
print(filter_df_a.shape)

(719, 28)


In [261]:
filter_df_a.to_csv("filter_a.csv", index=False)

In [301]:
df_all['交易筆棟數'].head()

0    土地2建物1車位0
1    土地2建物1車位0
2    土地1建物1車位0
3    土地2建物1車位0
4    土地1建物1車位1
Name: 交易筆棟數, dtype: object

In [308]:
# filter_b.csv
total_case_num = df_all.shape[0]
car_avg_price = int(df_all['車位總價元'].mean())
total_avg_price = int(df_all['總價元'].mean())
total_car_place = sum([int(transaction[-1]) for transaction in df_all['交易筆棟數'].values])

In [313]:
filter_df_b = pd.DataFrame({
    "總件數": [total_case_num],
    "總車位數": [total_car_place],
    "平均總價元": [total_avg_price],
    "平均車位總價元": [car_avg_price]
})

In [314]:
filter_df_b

,總件數,總車位數,平均總價元,平均車位總價元
0,4636,2247,12911140,396909


In [315]:
filter_df_b.to_csv("filter_b.csv", index=False)